In [64]:
#!/usr/bin/env python3

%matplotlib qt
import numpy as np
import pandas as pd
# import torch
# from dgl.geometry import farthest_point_sampler
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


In [65]:
df = pd.read_csv('/home/julius/Desktop/viewpoint_sampling/misc/poses_6d.csv')
#print(df.head())

# min = df['z'].abs().max()
# min_idx = df.index[df['z'] == min]
# print(min)
# print(min_idx.tolist())
# print(df.loc[min_idx.tolist()])


samples = 3

In [66]:
# x = torch.from_numpy(df.values)
# x = x.unsqueeze(0) 
# # print(x.shape)
# # print(type(x[0]))

# # print(x.shape)
# # y = torch.rand((2, 10, 3))
# # print(y)
# # print(y.shape)
# point_idx = farthest_point_sampler(x, samples)#, start_idx=0)
# point_idx = point_idx.flatten().tolist()
# print(point_idx)

# farthest_points = df.loc[point_idx]
# #farthest_points.sample(2)
# print(farthest_points['x'].values)


In [67]:
def fps(points, n_samples):
    """
    points: [N, 3] array containing the whole point cloud
    n_samples: samples you want in the sampled point cloud typically << N 
    """
    points = np.array(points)
    
    # Represent the points by their indices in points
    points_left = np.arange(len(points)) # [scipy_points]

    # Initialise an array for the sampled indices
    sample_inds = np.zeros(n_samples, dtype='int') # [S]

    # Initialise distances to inf
    dists = np.ones_like(points_left) * float('inf') # [scipy_points]

    # Select a point from points by its index, save it
    selected = 216
    sample_inds[0] = points_left[selected]

    # Delete selected 
    points_left = np.delete(points_left, selected) # [scipy_points - 1]

    # Iteratively select points for a maximum of n_samples
    for i in range(1, n_samples):
        # Find the distance to the last added point in selected
        # and all the others
        last_added = sample_inds[i-1]
        
        dist_to_last_added_point = (
            (points[last_added] - points[points_left])**2).sum(-1) # [scipy_points - i]

        # If closer, updated distances
        dists[points_left] = np.minimum(dist_to_last_added_point, 
                                        dists[points_left]) # [scipy_points - i]

        # We want to pick the one that has the largest nearest neighbour
        # distance to the sampled points
        selected = np.argmax(dists[points_left])
        sample_inds[i] = points_left[selected]

        # Update points_left
        points_left = np.delete(points_left, selected)

    return points[sample_inds]


In [68]:
values = df[['x', 'y', 'z']].to_numpy()
pnts = fps(values, samples)


In [69]:
import numpy as np
from scipy.spatial import ConvexHull
from scipy.spatial.distance import cdist

p = samples
# N = 16000000

# # Find a convex hull in O(N log N)
# points = np.random.rand(N, 3)   # N random points in 3-D
points = values
# # Returned 420 points in testing
# hull = ConvexHull(points)

# # Extract the points forming the hull
# hullpoints = points[hull.vertices,:]
hullpoints = values
# Naive way of finding the best pair in O(H^2) time if H is number of points on
# hull
hdist = cdist(hullpoints, hullpoints, metric='euclidean')
# print(hdist)
# print(hdist.shape)
# Get the farthest apart points
bestpair = np.unravel_index(hdist.argmax(), hdist.shape)

#print(bestpair.shape)

P = np.array([hullpoints[bestpair[0]],hullpoints[bestpair[1]]])

print("P:", P)
# Now we have a problem
print("Finding optimal set...")
while len(P)<p:
  print("P size = {0}".format(len(P)))
  distance_to_P = cdist(points, P, metric='euclidean')
  #print("distance_to_P", distance_to_P)
  print(distance_to_P.shape)
  minimum_to_each_of_P = np.min(distance_to_P, axis=1)
  best_new_point_idx   = np.argmax(minimum_to_each_of_P)
  best_new_point = np.expand_dims(points[best_new_point_idx,:],0)
  P = np.append(P,best_new_point,axis=0)


P: [[-0.3512 -0.241   0.83  ]
 [-0.5288  0.621   0.83  ]]
Finding optimal set...
P size = 2
(424, 2)


In [70]:
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(projection='3d')

ax.scatter(df['x'].values, df['y'].values, df['z'].values,  color='r', s=3, marker='X')
#ax.plot(farthest_points['x'].values, farthest_points['y'].values, farthest_points['z'].values, marker='o', markersize=5, color='black', label='Torch-Method')
#ax.scatter(P[:,0], P[:,1], P[:,2], marker='X', s=40, color='green', label='Scipy-Method')
ax.scatter(pnts[:,0], pnts[:,1], pnts[:,2], marker='*', s=40, color='blue', label=str(samples)+' samples')


#ax.plot(total_points[:, 0], total_points[:, 1], total_points[:, 2])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.legend()
ax.set_title("Farthest Point Sampling of Viewing Hemisphere")
plt.show()

In [71]:
rows_list = []
idx_list = np.array([])


for idx, subarr in enumerate(pnts):
    rows_list.append((df.loc[(df['x'] == subarr[0]) & (df['y'] == subarr[1]) & (df['z'] == subarr[2])]).values.flatten())
    idx_list = np.append(idx_list, ((df.index[(df['x'] == subarr[0]) & (df['y'] == subarr[1]) & (df['z'] == subarr[2])]).to_numpy())) 
    idx_list = idx_list.astype(int)
new_df = pd.DataFrame(columns= df.columns, data=rows_list, index=idx_list)
new_df.head(100)
#print(df.to_string())


,x,y,z,qx,qy,qz,qw
216,-0.4400,0.2201,1.269,-0.000000,0.999414,-0.034233,0.000000
9,-0.4099,-0.2490,0.830,0.706700,0.024197,-0.024198,-0.706685
118,-0.0029,0.2407,0.830,0.470301,0.528018,-0.528027,-0.470320


In [72]:


#for idx, img in enumerate(subsample_indices):
    
#subsample_indices = subsample_indices.zfill(6)
# print(subsample_indices)


# import shutil
# import os

# scenes_dir = "/home/julius/Documents/dataset_424/scenes"
# img_dir = "folder2"

# subfolders = sorted([f.path for f in os.scandir(scenes_dir) if f.is_dir()])

# print(subfolders)
# for subfolder in subfolders:
#     subfolder = subfolder[subfolder.index("0"):]
#     #print(subfolder)

#     new_dir = subfolder + '_samplesize_' + str(samples)
#     #print(new_dir)
#     new_dir_path = os.path.join("/home/julius/Documents/" + "dataset_" + str(samples), new_dir)
#     os.makedirs(new_dir_path, exist_ok=True)
#     new_dir_rgb = os.path.join(new_dir_path, 'rgb')
#     new_dir_depth = os.path.join(new_dir_path, 'depth')

#     src_dir_path = os.path.join(scenes_dir, subfolder)
#     src_dir_rgb = os.path.join(scenes_dir, subfolder + '/rgb')
#     src_dir_depth = os.path.join(scenes_dir, subfolder + '/depth')
#     os.makedirs(new_dir_rgb, exist_ok=True)
#     os.makedirs(new_dir_depth, exist_ok=True)
#     # os.mkdir(new_dir_rgb)
#     # os.mkdir(new_dir_depth)
#     print(src_dir_path)
#     print(src_dir_rgb)
#     print(src_dir_depth)
#     print("")
#     print(new_dir_path)
#     print(new_dir_rgb)
#     print(new_dir_depth)
#     print("")

#     shutil.copy(os.path.join(src_dir_path, 'associations.txt'), new_dir_path)
#     shutil.copy(os.path.join(src_dir_path, 'camera_d435.yaml'), new_dir_path)
#     shutil.copy(os.path.join(src_dir_path, 'groundtruth_handeye.txt'), new_dir_path)

#     # for imageName in subsample_indices:
#     #     shutil.copy(os.path.join(src_dir_rgb, imageName), new_dir_rgb)
#     #     shutil.copy(os.path.join(src_dir_depth, imageName), new_dir_depth)



In [73]:
import os
import shutil
import fnmatch

subsample_indices = new_df.index.to_list()
subsample_indices = [(str(item).zfill(6))+'.png' for item in subsample_indices]

dest = "/home/julius/Documents/" + "dataset_" + str(samples) + "/"
os.makedirs(dest, exist_ok=True)



for root, dirs, files in os.walk("/home/julius/Documents/dataset_424/scenes"):
    # print("root: ", root)
    # print("dirs: ", dirs)
    # print("files: ", files)
    # print("")
    for pattern in subsample_indices:
        for filename in fnmatch.filter(files, pattern):
            #print(filename)
            source = (os.path.join(root, filename))
            #print(source)
            destination = source.replace('424', str(samples))
            #print(destination)
            destination = os.path.dirname(destination) 
            os.makedirs(destination, exist_ok=True)
            #print(destination)
            if not os.path.exists(os.path.join(destination, filename)):
                shutil.copy2(source, destination)
                print("Copied {} from {} to {}".format(filename, source, destination))
                print("")

            else:
                print("File {} already exists in {}".format(filename, destination))

shutil.copy2("/home/julius/Documents/dataset_424/config.cfg", dest)
shutil.copytree("/home/julius/Documents/dataset_424/objects", dest+'/objects', dirs_exist_ok=True)

Copied 000216.png from /home/julius/Documents/dataset_424/scenes/001_standing_liquid_nerfacto_dex_nerf/depth/000216.png to /home/julius/Documents/dataset_3/scenes/001_standing_liquid_nerfacto_dex_nerf/depth

Copied 000009.png from /home/julius/Documents/dataset_424/scenes/001_standing_liquid_nerfacto_dex_nerf/depth/000009.png to /home/julius/Documents/dataset_3/scenes/001_standing_liquid_nerfacto_dex_nerf/depth

Copied 000118.png from /home/julius/Documents/dataset_424/scenes/001_standing_liquid_nerfacto_dex_nerf/depth/000118.png to /home/julius/Documents/dataset_3/scenes/001_standing_liquid_nerfacto_dex_nerf/depth

Copied 000216.png from /home/julius/Documents/dataset_424/scenes/001_standing_liquid_nerfacto_dex_nerf/rgb/000216.png to /home/julius/Documents/dataset_3/scenes/001_standing_liquid_nerfacto_dex_nerf/rgb

Copied 000009.png from /home/julius/Documents/dataset_424/scenes/001_standing_liquid_nerfacto_dex_nerf/rgb/000009.png to /home/julius/Documents/dataset_3/scenes/001_standin

'/home/julius/Documents/dataset_3//objects'